In [1]:
library(tidyr)
library(tidyverse) 
library(knitr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ dplyr   1.0.10
✔ tibble  3.1.8      ✔ stringr 1.4.1 
✔ readr   2.1.3      ✔ forcats 0.5.2 
✔ purrr   0.3.4      
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


# Data Cleaning/Import

In [2]:
# Read in CSV file
test_data <- read.csv("R_test.csv")

# Remove second row so column names are preserved
test_data <- test_data[-c(1),]

# Change column names
colnames(test_data)[1] <- "Record_ID"
colnames(test_data)[2] <- "email"

# Convert "Unchecked" and empty values to NA in all columns
checked_columns <- colnames(test_data)[-c(1, 2)]  # Get column names excluding Record_ID and email

for (col in checked_columns) {
  test_data[[col]] <- ifelse(test_data[[col]] %in% c("Unchecked", ""), NA, test_data[[col]])
}

# Remove rows with all missing values - remove rows where all values are NA except for "Record_ID", "email", and "Q219" columns
test_data <- test_data[rowSums(is.na(test_data[, 3:844])) != (844 - 3 + 1), ]

# Find rows with duplicate email
duplicate_data <- test_data[duplicated(test_data$email) | duplicated(test_data$email, fromLast = TRUE), ]

# Merge rows with duplicate email, keeping only non-missing values and updating Record_ID
merged_data <- duplicate_data %>%
  group_by(email) %>%
  summarize(across(everything(), ~ ifelse(all(is.na(.)), NA, first(na.omit(.)))), .groups = "drop")

# Combine non-duplicate rows with merged rows
final_data <- test_data %>%
  group_by(email) %>%
  summarize(across(everything(), ~ ifelse(all(is.na(.)), NA, first(na.omit(.)))), .groups = "drop")

# Convert "Checked" to 1
final_data[] <- lapply(final_data, function(x) ifelse(x == "Checked", 1, x))

In [3]:
# preview of cleaned data
head(final_data, 10)

email,Record_ID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,⋯,Q810,Q811,Q812,Q813,Q814,Q815,Q816,Q817,Q818,Q819
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
,1136,LBGTQ Health Center/Community Center,NA,Connecticut,6511,Urban,Yes,1,1,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,Incomplete
19tyler@gmail.com,422,Non-profit organization,NA,District of Columbia,20005,Urban,No,1,NA,⋯,fear of how far laws will go and stigmatizingp -,Unsure,fear,No,NA,Yes,Yes,gov refuses to use evidenced based data,"limitations on providing harm Reduction services, syringe exchanges and safe injection sites in heavily using areas.",Complete
3sollenberger@gmail.com,1294,Non-profit organization,NA,Pennsylvania,15222,Urban,No,1,1,⋯,NA,Yes,NA,No,NA,No,NA,NA,NA,Complete
6ap464zdb@mozmail.com,687,Community Based Organization (CBO),NA,New York,14527,Rural,No,1,1,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,Incomplete
aabundiz@hyacinth.org,1519,Community Based Organization (CBO),NA,New Jersey,7060,Suburban,No,NA,NA,⋯,NA,Unsure,NA,No,NA,No,NA,NA,NA,Complete
aadegbulugbe@totalhealthcare.org,222,Federally Qualified Health Center (FQHC),NA,Maryland,21223,Urban,Yes,1,1,⋯,NA,Yes,increases stress due to more stigma and discrimination,No,NA,No,NA,NA,"Major issue is with insurance coverage for medications which has no rhyme or reason and significantly disrupts patient care and results in major increase in workload, need coverage protections for ART and PrEP medications returned",Complete
aalbior@bach.health,1550,Federally Qualified Health Center (FQHC),NA,California,94538,Suburban,No,NA,NA,⋯,NA,Yes,NA,No,NA,No,NA,NA,NA,Complete
aalcanta@conemaugh.org,1385,AIDS Service Organization (ASO),NA,Pennsylvania,15905,Rural,No,1,1,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,Incomplete
abby.hinz@eauclairecounty.gov,258,Health department clinic/public health clinic,NA,Wisconsin,54703,Urban,Yes,1,NA,⋯,NA,Yes,Feel less support for the general community/society,No,NA,No,NA,NA,NA,Complete


## Number of incomplete and complete responses

In [4]:
# Complete vs incomplete for general data
table(final_data$Q819)


  Complete Incomplete 
       819        295 

# Descriptive Statistics for overall data

## Age counts

In [5]:
# Age
gen_age <- table(final_data$Q162)
gen_age


                18-24                 25-34                 35-44 
                   31                   204                   256 
                45-54                 55-64                    65 
                  262                   226                   103 
Prefer not to respond 
                   23 

## Age percentages

In [6]:
# Age
gen_age_perc <- prop.table(gen_age)*100
gen_age_perc


                18-24                 25-34                 35-44 
             2.805430             18.461538             23.167421 
                45-54                 55-64                    65 
            23.710407             20.452489              9.321267 
Prefer not to respond 
             2.081448 

## Race counts

In [7]:
## Race
gen_race <- table(final_data$Q159)
gen_race


        Asian and/or Asian American           Black or African American 
                                 28                                 273 
                        Multiracial    Native American or Alaska Native 
                                 79                                  13 
Native Hawaiian or Pacific Islander               Prefer not to respond 
                                  5                                  55 
            Prefer to self-describe                  White or Caucasian 
                                 26                                 625 

## Race percentages

In [8]:
# Race
gen_race_perc <- prop.table(gen_race)*100
gen_race_perc


        Asian and/or Asian American           Black or African American 
                          2.5362319                          24.7282609 
                        Multiracial    Native American or Alaska Native 
                          7.1557971                           1.1775362 
Native Hawaiian or Pacific Islander               Prefer not to respond 
                          0.4528986                           4.9818841 
            Prefer to self-describe                  White or Caucasian 
                          2.3550725                          56.6123188 

## Hispanic/Latinx counts

In [9]:
## Hispanic/Latinx
gen_hisp <- table(final_data$Q161)
gen_hisp



                   No Prefer not to respond                   Yes 
                  887                    21                   197 

## Hispanic/Latinx percentages

In [10]:
gen_hisp_perc <- prop.table(gen_hisp)*100
gen_hisp_perc


                   No Prefer not to respond                   Yes 
            80.271493              1.900452             17.828054 

## Type of organization counts

In [11]:
## Org type
gen_org_type <- table(final_data$Q1)
gen_org_type


             Academic hospital/clinic setting 
                                           80 
                        Advocacy organization 
                                            5 
              AIDS Service Organization (ASO) 
                                          133 
           Community Based Organization (CBO) 
                                          129 
     Federally Qualified Health Center (FQHC) 
                                          134 
    Government entity (Local, State, Federal) 
                                          105 
               Health department (non-clinic) 
                                           41 
Health department clinic/public health clinic 
                                          146 
         LBGTQ Health Center/Community Center 
                                           55 
                      Non-profit organization 
                                          159 
                                        Other 
            

## Type of organization percentages

In [12]:
gen_org_perc <- prop.table(gen_org_type)*100
gen_org_perc


             Academic hospital/clinic setting 
                                    7.1813285 
                        Advocacy organization 
                                    0.4488330 
              AIDS Service Organization (ASO) 
                                   11.9389587 
           Community Based Organization (CBO) 
                                   11.5798923 
     Federally Qualified Health Center (FQHC) 
                                   12.0287253 
    Government entity (Local, State, Federal) 
                                    9.4254937 
               Health department (non-clinic) 
                                    3.6804309 
Health department clinic/public health clinic 
                                   13.1059246 
         LBGTQ Health Center/Community Center 
                                    4.9371634 
                      Non-profit organization 
                                   14.2728905 
                                        Other 
            

## State counts

In [13]:
## States
gen_state_counts <- table(final_data$Q3)
gen_state_counts


             Alabama               Alaska              Arizona 
                   8                    1                   15 
            Arkansas           California             Colorado 
                   8                  105                   16 
         Connecticut             Delaware District of Columbia 
                  20                   12                   39 
             Florida              Georgia               Hawaii 
                  64                   26                   11 
               Idaho             Illinois              Indiana 
                   2                   38                   46 
                Iowa               Kansas             Kentucky 
                  13                    4                   30 
           Louisiana                Maine             Maryland 
                  24                    1                   67 
       Massachusetts             Michigan            Minnesota 
                   7                   

## State percentages

In [14]:
gen_state_perc <- prop.table(gen_state_counts)*100
gen_state_perc


             Alabama               Alaska              Arizona 
          0.72072072           0.09009009           1.35135135 
            Arkansas           California             Colorado 
          0.72072072           9.45945946           1.44144144 
         Connecticut             Delaware District of Columbia 
          1.80180180           1.08108108           3.51351351 
             Florida              Georgia               Hawaii 
          5.76576577           2.34234234           0.99099099 
               Idaho             Illinois              Indiana 
          0.18018018           3.42342342           4.14414414 
                Iowa               Kansas             Kentucky 
          1.17117117           0.36036036           2.70270270 
           Louisiana                Maine             Maryland 
          2.16216216           0.09009009           6.03603604 
       Massachusetts             Michigan            Minnesota 
          0.63063063           3.873873

## Role counts

In [15]:
gen_role_counts <- table(final_data$Q66)
gen_role_counts


                             Dentist               Dietician/Nutritionist 
                                   2                                    1 
     Disease Intervention Specialist      Licensed Clinical Social Worker 
                                   1                                   50 
            Licensed Practical Nurse                    Medical Assistant 
                                  10                                   10 
          Mental Health Case Manager              Mental Health Counselor 
                                   5                                    1 
          Mental Health Professional          Mental Health Professional  
                                  15                                    1 
                       Nurse Manager                        Nurse Midwife 
                                  19                                    1 
                  Nurse Practitioner Other Provider/Clinical professional 
                        

## Role percentages

In [16]:
gen_role_perc <- prop.table(gen_role_counts)*100
gen_role_perc


                             Dentist               Dietician/Nutritionist 
                           0.5277045                            0.2638522 
     Disease Intervention Specialist      Licensed Clinical Social Worker 
                           0.2638522                           13.1926121 
            Licensed Practical Nurse                    Medical Assistant 
                           2.6385224                            2.6385224 
          Mental Health Case Manager              Mental Health Counselor 
                           1.3192612                            0.2638522 
          Mental Health Professional          Mental Health Professional  
                           3.9577836                            0.2638522 
                       Nurse Manager                        Nurse Midwife 
                           5.0131926                            0.2638522 
                  Nurse Practitioner Other Provider/Clinical professional 
                        

## Prescriber counts

In [17]:
gen_prescriber_counts <- table(final_data$Q70)
gen_prescriber_counts


 No Yes 
229 154 

## Prescriber percentage

In [18]:
gen_prescriber_perc <- prop.table(gen_prescriber_counts)*100
gen_prescriber_perc


      No      Yes 
59.79112 40.20888 

# HIV training statistics for overall data

In [19]:
# Create a vector of column names representing the HIV topics columns
hiv_topics <- c("Q198", "Q199", "Q200", "Q201", "Q202","Q203","Q204","Q205","Q206","Q207","Q208","Q209","Q210a","Q210b")

## HIV topic with greatest number of 1st rankings

In [20]:
# Calculate the counts of 1st rankings for each HIV topic
hiv1_counts <- sapply(hiv_topics, function(topic) sum(final_data[[topic]] == 1, na.rm = TRUE))
hiv1_counts

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
  181    83    76    18    21    37    34    49    49     6    12    12     3 
Q210b 
    1

In [21]:
# Find the HIV topic with the highest count of 1st rankings
hiv1 <- hiv_topics[which.max(hiv1_counts)]
hiv1

[1] "Q198"

In [22]:
# Calculate the percentage of 1st rankings
hiv1_perc <- (hiv1_counts[which.max(hiv1_counts)] / nrow(final_data)) * 100
hiv1_perc

Q198 
16.24776

## HIV topic with greatest number of 2nd rankings

In [23]:
# Calculate the counts of 2nd rankings for each HIV topic
hiv2_counts <- sapply(hiv_topics, function(topic) sum(final_data[[topic]] == 2, na.rm = TRUE))
hiv2_counts

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
   61    89    76    27    44    52    48    49    56    15    19     2     0 
Q210b 
    1

In [24]:
# Find the HIV topic with the highest count of 2nd rankings
hiv2 <- hiv_topics[which.max(hiv2_counts)]
hiv2

[1] "Q199"

In [25]:
# Calculate the percentage of 2nd rankings for the top HIV topic
hiv2_perc <- (hiv2_counts[which.max(hiv2_counts)] / nrow(final_data)) * 100
hiv2_perc

Q199 
7.989228

## HIV topic with greatest number of 3rd rankings

In [26]:
# Calculate the counts of 3rd rankings for each HIV topic
hiv3_counts <- sapply(hiv_topics, function(topic) sum(final_data[[topic]] == 3, na.rm = TRUE))
hiv3_counts

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
   67    47    56    17    56    46    61    58    53    12    37    16     2 
Q210b 
    3

In [27]:
# Find the HIV topic with the highest count of 3rd rankings
hiv3 <- hiv_topics[which.max(hiv3_counts)]
hiv3

[1] "Q198"

In [28]:
# Calculate the percentage of 3rd rankings
hiv3_perc <- (hiv3_counts[which.max(hiv3_counts)] / nrow(final_data)) * 100
hiv3_perc

Q198 
6.014363

## Top 3 HIV training topics

In [29]:
# Calculate the counts for each HIV topic
topic_counts <- sapply(hiv_topics, function(topic) sum(topic %in% c(1, 2, 3), na.rm = TRUE))

# Sort the topics based on their counts in descending order
sorted_topics <- sort(topic_counts, decreasing = TRUE)

# Get the top 3 topics
top_3_topics <- names(sorted_topics)[1:3]

top_3_topics

[1] "Q198" "Q199" "Q200"

# PrEP training statistics for overall data

In [30]:
# Create a vector of column names representing the PrEP topics columns
prep_topics <- c("Q211", "Q212", "Q213", "Q214", "Q215","Q216","Q217","Q218","Q219","Q220","Q221","Q222","Q223","Q224","Q225")

## PrEP topic with greatest number of 1st rankings

In [31]:
# Calculate the counts of 1st rankings for each PrEP topic
prep1_counts <- sapply(prep_topics, function(topic) sum(final_data[[topic]] == 1, na.rm = TRUE))
prep1_counts

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
 124   73   62  108   11   22   20   48   37   31   19   29   36   12    3

In [32]:
# Find the PrEP topic with the highest count of 1st rankings
prep1 <- prep_topics[which.max(prep1_counts)]
prep1

[1] "Q211"

In [33]:
# Calculate the percentage of 1st rankings
prep1_perc <- (prep1_counts[which.max(prep1_counts)] / nrow(final_data)) * 100
prep1_perc

Q211 
11.13106

## PrEP topic with greatest number of 2nd rankings

In [34]:
# Calculate the counts of 2nd rankings for each PrEP topic
prep2_counts <- sapply(prep_topics, function(topic) sum(final_data[[topic]] == 2, na.rm = TRUE))
prep2_counts

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
  29   70   53   80   26   34   37   77   50   39   24   27   41   15    0

In [35]:
# Find the PrEP topic with the highest count of 2nd rankings
prep2 <- prep_topics[which.max(prep2_counts)]
prep2

[1] "Q214"

In [36]:
# Calculate the percentage of 2nd rankings for the top PrEP topic
prep2_perc <- (prep2_counts[which.max(prep2_counts)] / nrow(final_data)) * 100
prep2_perc

Q214 
7.181329

## PrEP topic with greatest number of 3rd rankings

In [37]:
# Calculate the counts of 3rd rankings for each PrEP topic
prep3_counts <- sapply(prep_topics, function(topic) sum(final_data[[topic]] == 3, na.rm = TRUE))
prep3_counts

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
  44   21   43   52   18   35   38   69   26   66   42   46   42   41    5

In [38]:
# Find the PrEP topic with the highest count of 3rd rankings
prep3 <- prep_topics[which.max(prep3_counts)]
prep3

[1] "Q218"

In [39]:
# Calculate the percentage of 3rd rankings
prep3_perc <- (prep3_counts[which.max(prep3_counts)] / nrow(final_data)) * 100
prep3_perc

Q218 
6.193896

## Top 3 PrEP training topics

In [40]:
# Calculate the counts for each PrEP topic
topic_counts <- sapply(prep_topics, function(topic) sum(topic %in% c(1, 2, 3), na.rm = TRUE))

# Sort the topics based on their counts in descending order
sorted_topics <- sort(topic_counts, decreasing = TRUE)

# Get the top 3 topics
top_3_topics <- names(sorted_topics)[1:3]

top_3_topics

[1] "Q211" "Q212" "Q213"

# LGBTQ training statistics for overall data

In [41]:
# Create a vector of column names representing the LGBTQ topics columns
lgbtq_topics <- c("Q227", "Q228", "Q229", "Q230", "Q231","Q232","Q233","Q234","Q235")

## LGBTQ topic with greatest number of 1st rankings

In [42]:
# Calculate the counts of 1st rankings for each LGBTQ topic
lgbtq1_counts <- sapply(lgbtq_topics, function(topic) sum(final_data[[topic]] == 1, na.rm = TRUE))
lgbtq1_counts

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
 139  198  121  120   34   67  118  114   15

In [43]:
# Find the LGBTQ topic with the highest count of 1st rankings
lgbtq1 <- lgbtq_topics[which.max(lgbtq1_counts)]
lgbtq1

[1] "Q228"

In [44]:
# Calculate the percentage of 1st rankings
lgbtq1_perc <- (lgbtq1_counts[which.max(lgbtq1_counts)] / nrow(final_data)) * 100
lgbtq1_perc

Q228 
17.77379

## LGBTQ topic with greatest number of 2nd rankings

In [45]:
# Calculate the counts of 2nd rankings for each LGBTQ topic
lgbtq2_counts <- sapply(lgbtq_topics, function(topic) sum(final_data[[topic]] == 2, na.rm = TRUE))
lgbtq2_counts

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
  67  121  102  154   62   78  143  143    4

In [46]:
# Find the LGBTQ topic with the highest count of 2nd rankings
lgbtq2 <- lgbtq_topics[which.max(lgbtq2_counts)]
lgbtq2

[1] "Q230"

In [47]:
# Calculate the percentage of 2nd rankings for the top LGBTQ topic
lgbtq2_perc <- (lgbtq2_counts[which.max(lgbtq2_counts)] / nrow(final_data)) * 100
lgbtq2_perc

Q230 
13.82406

## LGBTQ topic with greatest number of 3rd rankings

In [48]:
# Calculate the counts of 3rd rankings for each LGBTQ topic
lgbtq3_counts <- sapply(lgbtq_topics, function(topic) sum(final_data[[topic]] == 3, na.rm = TRUE))
lgbtq3_counts

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
  81  122   73  124   66   64  109  194   14

In [49]:
# Find the LGBTQ topic with the highest count of 3rd rankings
lgbtq3 <- lgbtq_topics[which.max(lgbtq3_counts)]
lgbtq3

[1] "Q234"

In [50]:
# Calculate the percentage of 3rd rankings
lgbtq3_perc <- (lgbtq3_counts[which.max(lgbtq3_counts)] / nrow(final_data)) * 100
lgbtq3_perc

Q234 
17.41472

## Top 3 LGBTQ training topics 

In [51]:
# Calculate the counts for each LGBTQ topic
topic_counts <- sapply(lgbtq_topics, function(topic) sum(topic %in% c(1, 2, 3), na.rm = TRUE))

# Sort the topics based on their counts in descending order
sorted_topics <- sort(topic_counts, decreasing = TRUE)

# Get the top 3 topics
top_3_topics <- names(sorted_topics)[1:3]

top_3_topics

[1] "Q227" "Q228" "Q229"

# Identifying HIV providers

In [52]:
# HIV providers
HIV_providers <- final_data[final_data$Q22 == "Yes" | final_data$Q30 == "Yes" | final_data$Q32 == "Yes", ]
head(HIV_providers, 10)

email,Record_ID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,⋯,Q810,Q811,Q812,Q813,Q814,Q815,Q816,Q817,Q818,Q819
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
19tyler@gmail.com,422,Non-profit organization,NA,District of Columbia,20005,Urban,No,1,NA,⋯,fear of how far laws will go and stigmatizingp -,Unsure,fear,No,NA,Yes,Yes,gov refuses to use evidenced based data,"limitations on providing harm Reduction services, syringe exchanges and safe injection sites in heavily using areas.",Complete
3sollenberger@gmail.com,1294,Non-profit organization,NA,Pennsylvania,15222,Urban,No,1,1,⋯,NA,Yes,NA,No,NA,No,NA,NA,NA,Complete
aabundiz@hyacinth.org,1519,Community Based Organization (CBO),NA,New Jersey,7060,Suburban,No,NA,NA,⋯,NA,Unsure,NA,No,NA,No,NA,NA,NA,Complete
aadegbulugbe@totalhealthcare.org,222,Federally Qualified Health Center (FQHC),NA,Maryland,21223,Urban,Yes,1,1,⋯,NA,Yes,increases stress due to more stigma and discrimination,No,NA,No,NA,NA,"Major issue is with insurance coverage for medications which has no rhyme or reason and significantly disrupts patient care and results in major increase in workload, need coverage protections for ART and PrEP medications returned",Complete
aalbior@bach.health,1550,Federally Qualified Health Center (FQHC),NA,California,94538,Suburban,No,NA,NA,⋯,NA,Yes,NA,No,NA,No,NA,NA,NA,Complete
aalcanta@conemaugh.org,1385,AIDS Service Organization (ASO),NA,Pennsylvania,15905,Rural,No,1,1,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,Incomplete
abby.hinz@eauclairecounty.gov,258,Health department clinic/public health clinic,NA,Wisconsin,54703,Urban,Yes,1,NA,⋯,NA,Yes,Feel less support for the general community/society,No,NA,No,NA,NA,NA,Complete
abilly@iuhealth.org,774,AIDS Service Organization (ASO),NA,Indiana,47909,Urban,No,1,1,⋯,NA,Yes,NA,No,NA,No,NA,NA,NA,Complete


## Number of incomplete and complete responses among HIV providers

In [53]:
# Complete vs Incomplete among HIV providers
table(HIV_providers$Q819)


  Complete Incomplete 
       760        271 

## Descriptive Statistics among HIV providers

### Age counts among HIV providers

In [54]:
## HIV providers - Age
age_HIV <- table(HIV_providers$Q162)
age_HIV


                18-24                 25-34                 35-44 
                   31                   196                   239 
                45-54                 55-64                    65 
                  249                   204                    91 
Prefer not to respond 
                   19 

### Age percentages among HIV providers

In [55]:
age_perc_HIV <- prop.table(age_HIV)*100
age_perc_HIV


                18-24                 25-34                 35-44 
             3.012634             19.047619             23.226433 
                45-54                 55-64                    65 
            24.198251             19.825073              8.843537 
Prefer not to respond 
             1.846453 

### Gender counts among HIV providers

In [56]:
## HIV providers - Gender
gender_HIV<- table(HIV_providers$Q157)
gender_HIV


                                Female/Woman 
                                         636 
Genderqueer/gender non-confirming/non-binary 
                                          64 
                                    Male/Man 
                                         271 
                       Prefer not to respond 
                                          17 
                     Prefer to self-describe 
                                           9 
                             Transgender man 
                                          18 
                           Transgender woman 
                                          14 

### Gender percentages among HIV providers

In [57]:
gender_perc_HIV <- prop.table(gender_HIV)*100
gender_perc_HIV


                                Female/Woman 
                                  61.8075802 
Genderqueer/gender non-confirming/non-binary 
                                   6.2196307 
                                    Male/Man 
                                  26.3362488 
                       Prefer not to respond 
                                   1.6520894 
                     Prefer to self-describe 
                                   0.8746356 
                             Transgender man 
                                   1.7492711 
                           Transgender woman 
                                   1.3605442 

### Race counts among HIV providers

In [58]:
## HIV providers - Race
race_HIV <- table(HIV_providers$Q159)
race_HIV


        Asian and/or Asian American           Black or African American 
                                 26                                 250 
                        Multiracial    Native American or Alaska Native 
                                 77                                  11 
Native Hawaiian or Pacific Islander               Prefer not to respond 
                                  5                                  52 
            Prefer to self-describe                  White or Caucasian 
                                 25                                 582 

### Race percentages among HIV providers

In [59]:
race_perc_HIV <- prop.table(race_HIV)*100
race_perc_HIV


        Asian and/or Asian American           Black or African American 
                          2.5291829                          24.3190661 
                        Multiracial    Native American or Alaska Native 
                          7.4902724                           1.0700389 
Native Hawaiian or Pacific Islander               Prefer not to respond 
                          0.4863813                           5.0583658 
            Prefer to self-describe                  White or Caucasian 
                          2.4319066                          56.6147860 

### Hispanic/Latinx counts among HIV providers

In [60]:
## HIV providers - Hispanic/Latinx
hisp_HIV <- table(HIV_providers$Q161)
hisp_HIV


                   No Prefer not to respond                   Yes 
                  819                    19                   191 

### Hispanic/Latinx percentages among HIV providers

In [61]:
hisp_perc_HIV<- prop.table(hisp_HIV)*100
hisp_perc_HIV


                   No Prefer not to respond                   Yes 
            79.591837              1.846453             18.561710 

### Role counts among HIV providers

In [62]:
role_counts_HIV <- table(HIV_providers$Q66)
role_counts_HIV


                             Dentist      Disease Intervention Specialist 
                                   2                                    1 
     Licensed Clinical Social Worker             Licensed Practical Nurse 
                                  37                                   10 
                   Medical Assistant           Mental Health Case Manager 
                                  10                                    3 
             Mental Health Counselor           Mental Health Professional 
                                   1                                   11 
         Mental Health Professional                         Nurse Manager 
                                   1                                   18 
                       Nurse Midwife                   Nurse Practitioner 
                                   1                                   68 
Other Provider/Clinical professional                           Pharmacist 
                        

### Role percentages among HIV providers

In [63]:
role_perc_HIV <- prop.table(role_counts_HIV)*100
role_perc_HIV


                             Dentist      Disease Intervention Specialist 
                           0.5830904                            0.2915452 
     Licensed Clinical Social Worker             Licensed Practical Nurse 
                          10.7871720                            2.9154519 
                   Medical Assistant           Mental Health Case Manager 
                           2.9154519                            0.8746356 
             Mental Health Counselor           Mental Health Professional 
                           0.2915452                            3.2069971 
         Mental Health Professional                         Nurse Manager 
                           0.2915452                            5.2478134 
                       Nurse Midwife                   Nurse Practitioner 
                           0.2915452                           19.8250729 
Other Provider/Clinical professional                           Pharmacist 
                        

### Prescriber counts among HIV providers

In [64]:
prescriber_counts_HIV <- table(HIV_providers$Q70)
prescriber_counts_HIV


 No Yes 
198 149 

### Prescriber percentages among HIV providers

In [65]:
prescriber_perc_HIV <- prop.table(prescriber_counts_HIV)*100
prescriber_perc_HIV


      No      Yes 
57.06052 42.93948 

## HIV training statistics for HIV providers

In [66]:
# HIV providers - Create a vector of column names representing the HIV topics columns
hiv_topics_HIV <- c("Q198", "Q199", "Q200", "Q201", "Q202","Q203","Q204","Q205","Q206","Q207","Q208","Q209","Q210a","Q210b")

### HIV topic with greatest number of 1st rankings among HIV providers

In [67]:
# HIV providers - Calculate the counts of 1st rankings for each HIV topic
hiv1_count_HIV <- sapply(hiv_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 1, na.rm = TRUE))
hiv1_count_HIV

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
  181    83    76    18    21    37    34    49    49     6    12    12     3 
Q210b 
    1

In [68]:
# HIV providers - Find the HIV topic with the highest count of 1st rankings
hiv1_rank_HIV <- hiv_topics_HIV[which.max(hiv1_count_HIV)]
hiv1_rank_HIV

[1] "Q198"

In [69]:
# HIV providers - Calculate the percentage of 1st rankings
hiv1_perc_HIV <- (hiv1_count_HIV[which.max(hiv1_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, hiv_topics_HIV])) > 0)) * 100
hiv1_perc_HIV

Q198 
30.83475

### HIV topic with greatest number of 2nd rankings among HIV providers

In [70]:
# HIV providers - Calculate the counts of 2nd rankings for each HIV topic
hiv2_count_HIV <- sapply(hiv_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 2, na.rm = TRUE))
hiv2_count_HIV

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
   61    89    76    27    44    52    48    49    56    15    19     2     0 
Q210b 
    1

In [71]:
# HIV providers - Find the HIV topic with the highest count of 2nd rankings
hiv2_rank_HIV <- hiv_topics_HIV[which.max(hiv2_count_HIV)]
hiv2_rank_HIV

[1] "Q199"

In [72]:
# HIV providers - Calculate the percentage of 2nd ranking
hiv2_perc_HIV <- (hiv2_count_HIV[which.max(hiv2_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, hiv_topics_HIV])) > 0)) * 100
hiv2_perc_HIV

Q199 
15.16184

### HIV topic with greatest number of 3rd rankings among HIV providers

In [73]:
# HIV providers - Calculate the counts of 3rd rankings for each HIV topic
hiv3_count_HIV <- sapply(hiv_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 3, na.rm = TRUE))
hiv3_count_HIV

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
   67    47    56    17    56    46    61    58    53    12    37    16     2 
Q210b 
    3

In [74]:
# HIV providers - Find the HIV topic with the highest count of 3rd rankings
hiv3_rank_HIV <- hiv_topics_HIV[which.max(hiv3_count_HIV)]
hiv3_rank_HIV

[1] "Q198"

In [75]:
# HIV providers - Calculate the percentage of 3rd rankings
hiv3_perc_HIV <- (hiv3_count_HIV[which.max(hiv3_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, hiv_topics_HIV])) > 0)) * 100
hiv3_perc_HIV

Q198 
11.41397

### Top 3 HIV topics among HIV providers

In [76]:
# Calculate the counts for each HIV topic
hiv_topic_counts_HIV <- sapply(hiv_topics_HIV, function(topic) sum(HIV_providers[[topic]] %in% c(1, 2, 3), na.rm = TRUE))
hiv_topic_counts_HIV
# Sort the topics based on their counts in descending order
hiv_sorted_topics_HIV <- sort(hiv_topic_counts_HIV, decreasing = TRUE)
hiv_sorted_topics_HIV
# Get the top 3 topics
top3_hiv_topics_HIV <- names(hiv_sorted_topics_HIV)[1:3]

top3_hiv_topics_HIV

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
  309   219   208    62   121   135   143   156   158    33    68    30     5 
Q210b 
    5

Q198  Q199  Q200  Q206  Q205  Q204  Q203  Q202  Q208  Q201  Q207  Q209 Q210a 
  309   219   208   158   156   143   135   121    68    62    33    30     5 
Q210b 
    5

[1] "Q198" "Q199" "Q200"

### Percentage of top 3 HIV topics among HIV providers

In [77]:
# Calculate the counts for each ranking (1, 2, 3) for the top 3 topics
hiv_ranking_counts_HIV <- apply(HIV_providers[top3_hiv_topics_HIV], 1, function(row) any(row %in% c(1, 2, 3)))

# Calculate the total count of rows where one of the top 3 topics was ranked as 1, 2, or 3
hiv_total_count_HIV <- sum(hiv_ranking_counts_HIV)

# Calculate the total count of respondents who provided a ranking (1, 2, or 3) for any of the HIV topic questions
hiv_total_respondents_HIV <- sum(apply(HIV_providers[hiv_topics_HIV], 1, function(row) any(row %in% c(1, 2, 3))))

# Calculate the percentage of rows where one of the top 3 topics was ranked as 1, 2, or 3 among the respondents who provided rankings
hiv_total_percentage_HIV <- (hiv_total_count_HIV / hiv_total_respondents_HIV) * 100
hiv_total_percentage_HIV                                

[1] 81.43101

## Prep training statistics for HIV providers

In [78]:
# HIV providers - Create a vector of column names representing the PrEP topics columns
prep_topics_HIV <- c("Q211", "Q212", "Q213", "Q214", "Q215","Q216","Q217","Q218","Q219","Q220","Q221","Q222","Q223","Q224","Q225")

### PrEP topic with greatest number of 1st rankings among HIV providers

In [79]:
# HIV providers - Calculate the counts of 1st rankings for each PrEP topic
prep1_count_HIV <- sapply(prep_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 1, na.rm = TRUE))
prep1_count_HIV

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
 124   73   62  108   11   22   20   48   37   31   19   29   36   12    3

In [80]:
# HIV providers - Find the PrEP topic with the highest count of 1st rankings
prep1_rank_HIV <- prep_topics_HIV[which.max(prep1_count_HIV)]
prep1_rank_HIV

[1] "Q211"

In [81]:
# HIV providers - Calculate the percentage of 1st rankings
prep1_perc_HIV <- (prep1_count_HIV[which.max(prep1_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, prep_topics_HIV])) > 0)) * 100
prep1_perc_HIV

Q211 
19.2846

### PrEP topic with greatest number of 2nd rankings among HIV providers

In [82]:
# HIV providers - Calculate the counts of 2nd rankings for each PrEP topic
prep2_count_HIV <- sapply(prep_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 2, na.rm = TRUE))
prep2_count_HIV

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
  29   70   53   80   26   34   37   77   50   39   24   27   41   15    0

In [83]:
# HIV providers - Find the PrEP topic with the highest count of 2nd rankings
prep2_rank_HIV <- prep_topics_HIV[which.max(prep2_count_HIV)]
prep2_rank_HIV

[1] "Q214"

In [84]:
# HIV providers - Calculate the percentage of 2nd ranking
prep2_perc_HIV <- (prep2_count_HIV[which.max(prep2_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, prep_topics_HIV])) > 0)) * 100
prep2_perc_HIV

Q214 
12.44168

### PrEP topic with greatest number of 3rd rankings among HIV providers

In [85]:
# HIV providers - Calculate the counts of 3rd rankings for each PrEP topic
prep3_count_HIV <- sapply(prep_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 3, na.rm = TRUE))
prep3_count_HIV

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
  44   21   43   52   18   35   38   69   26   66   42   46   42   41    5

In [86]:
# HIV providers - Find the PrEP topic with the highest count of 3rd rankings
prep3_rank_HIV <- prep_topics_HIV[which.max(prep3_count_HIV)]
prep3_rank_HIV

[1] "Q218"

In [87]:
# HIV providers - Calculate the percentage of 3rd rankings
prep3_perc_HIV <- (prep3_count_HIV[which.max(prep3_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, prep_topics_HIV])) > 0)) * 100
prep3_perc_HIV

Q218 
10.73095

### Top 3 PrEP topics among HIV providers

In [88]:
# Calculate the counts for each PrEP topic
prep_topic_counts_HIV <- sapply(prep_topics_HIV, function(topic) sum(HIV_providers[[topic]] %in% c(1, 2, 3), na.rm = TRUE))

# Sort the topics based on their counts in descending order
prep_sorted_topics_HIV <- sort(prep_topic_counts_HIV, decreasing = TRUE)

# Get the top 3 topics
top3_prep_topics_HIV <- names(prep_sorted_topics_HIV)[1:3]

top3_prep_topics_HIV

[1] "Q214" "Q211" "Q218"

### Percentage of top 3 PrEP topics among HIV providers

In [89]:
# Calculate the counts for each ranking (1, 2, 3) for the top 3 topics
prep_ranking_counts_HIV <- apply(HIV_providers[top3_prep_topics_HIV], 1, function(row) any(row %in% c(1, 2, 3)))

# Calculate the total count of rows where one of the top 3 topics was ranked as 1, 2, or 3
prep_total_count_HIV <- sum(prep_ranking_counts_HIV)

# Calculate the total count of respondents who provided a ranking (1, 2, or 3) for any of the HIV topic questions
prep_total_respondents_HIV <- sum(apply(HIV_providers[prep_topics_HIV], 1, function(row) any(row %in% c(1, 2, 3))))

# Calculate the percentage of rows where one of the top 3 topics was ranked as 1, 2, or 3 among the respondents who provided rankings
prep_total_percentage_HIV <- (prep_total_count_HIV / prep_total_respondents_HIV) * 100
prep_total_percentage_HIV         

[1] 73.87247

## LGBTQ training statistcis for HIV providers

In [90]:
# Create a vector of column names representing the LGBTQ topics columns
lgbtq_topics_HIV <- c("Q227", "Q228", "Q229", "Q230", "Q231","Q232","Q233","Q234","Q235")

### LGBTQ topic with greatest number of 1st rankings among HIV providers

In [91]:
# HIV providers - Calculate the counts of 1st rankings for each LGBTQ topic
lgbtq1_count_HIV <- sapply(lgbtq_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 1, na.rm = TRUE))
lgbtq1_count_HIV

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
 132  186  113  110   29   63  112  109   11

In [92]:
# HIV providers - Find the LGBTQ topic with the highest count of 1st rankings
lgbtq1_rank_HIV <- lgbtq_topics_HIV[which.max(lgbtq1_count_HIV)]
lgbtq1_rank_HIV

[1] "Q228"

In [93]:
# HIV providers - Calculate the percentage of 1st rankings
lgbtq1_perc_HIV <- (lgbtq1_count_HIV[which.max(lgbtq1_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, lgbtq_topics_HIV])) > 0)) * 100
lgbtq1_perc_HIV

Q228 
21.20867

### LGBTQ topic with greatest number of 2nd rankings among HIV providers

In [94]:
# HIV providers - Calculate the counts of 2nd rankings for each LGBTQ topic
lgbtq2_count_HIV <- sapply(lgbtq_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 2, na.rm = TRUE))
lgbtq2_count_HIV

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
  63  114   99  142   58   73  128  134    3

In [95]:
# HIV providers - Find the LGBTQ topic with the highest count of 2nd rankings
lgbtq2_rank_HIV <- lgbtq_topics_HIV[which.max(lgbtq2_count_HIV)]
lgbtq2_rank_HIV

[1] "Q230"

In [96]:
# HIV providers - Calculate the percentage of 2nd rankings
lgbtq2_perc_HIV <- (lgbtq2_count_HIV[which.max(lgbtq2_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, lgbtq_topics_HIV])) > 0)) * 100
lgbtq2_perc_HIV

Q230 
16.19156

### LGBTQ topic with greatest number of 3rd rankings among HIV providers

In [97]:
# HIV providers - Calculate the counts of 3rd rankings for each LGBTQ topic
lgbtq3_count_HIV <- sapply(lgbtq_topics_HIV, function(topic) sum(HIV_providers[[topic]] == 3, na.rm = TRUE))
lgbtq3_count_HIV

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
  74  115   69  115   63   60  104  177   13

In [98]:
# HIV providers - Find the LGBTQ topic with the highest count of 3rd rankings
lgbtq3_rank_HIV <- lgbtq_topics_HIV[which.max(lgbtq3_count_HIV)]
lgbtq3_rank_HIV

[1] "Q234"

In [99]:
# HIV providers - Calculate the percentage of 3rd rankings
lgbtq3_perc_HIV <- (lgbtq3_count_HIV[which.max(lgbtq3_count_HIV)] / sum(rowSums(!is.na(HIV_providers[, lgbtq_topics_HIV])) > 0)) * 100
lgbtq3_perc_HIV

Q234 
20.18244

### Top 3 LGBTQ topics among HIV providers

In [100]:
# Calculate the counts for each LGBTQ topic
lgbtq_topic_counts_HIV <- sapply(lgbtq_topics_HIV, function(topic) sum(HIV_providers[[topic]] %in% c(1, 2, 3), na.rm = TRUE))

# Sort the topics based on their counts in descending order
lgbtq_sorted_topics_HIV <- sort(lgbtq_topic_counts_HIV, decreasing = TRUE)

# Get the top 3 topics
top3_lgbtq_topics_HIV <- names(lgbtq_sorted_topics_HIV)[1:3]
top3_lgbtq_topics_HIV

[1] "Q234" "Q228" "Q230"

### Percentage of top 3 LGBTQ topics among HIV providers

In [101]:
# Calculate the counts for each ranking (1, 2, 3) for the top 3 topics
lgbtq_ranking_counts_HIV <- apply(HIV_providers[top3_lgbtq_topics_HIV], 1, function(row) any(row %in% c(1, 2, 3)))

# Calculate the total count of rows where one of the top 3 topics was ranked as 1, 2, or 3
lgbtq_total_count_HIV <- sum(lgbtq_ranking_counts_HIV)

# Calculate the total count of respondents who provided a ranking (1, 2, or 3) for any of the LGBTQ topic questions
lgbtq_total_respondents_HIV <- sum(apply(HIV_providers[lgbtq_topics_HIV], 1, function(row) any(row %in% c(1, 2, 3))))

# Calculate the percentage of rows where one of the top 3 topics was ranked as 1, 2, or 3 among the respondents who provided rankings
lgbtq_total_percentage_HIV <- (lgbtq_total_count_HIV / lgbtq_total_respondents_HIV) * 100
lgbtq_total_percentage_HIV 

[1] 87.00114

## Identifying LGBTQ providers

In [102]:
# LGBTQ providers
LGBTQ_providers <- final_data[final_data$Q275 == "Yes" | final_data$Q130 == 1 | final_data$Q1 == "LBGTQ Health Center/Community Center" | final_data$Q22 == "Yes" | final_data$Q33 == 1, ]
head(LGBTQ_providers, 10)

email,Record_ID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,⋯,Q810,Q811,Q812,Q813,Q814,Q815,Q816,Q817,Q818,Q819
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
,1136,LBGTQ Health Center/Community Center,NA,Connecticut,6511,Urban,Yes,1,1,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,Incomplete
19tyler@gmail.com,422,Non-profit organization,NA,District of Columbia,20005,Urban,No,1,NA,⋯,fear of how far laws will go and stigmatizingp -,Unsure,fear,No,NA,Yes,Yes,gov refuses to use evidenced based data,"limitations on providing harm Reduction services, syringe exchanges and safe injection sites in heavily using areas.",Complete
3sollenberger@gmail.com,1294,Non-profit organization,NA,Pennsylvania,15222,Urban,No,1,1,⋯,NA,Yes,NA,No,NA,No,NA,NA,NA,Complete
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
aabundiz@hyacinth.org,1519,Community Based Organization (CBO),NA,New Jersey,7060,Suburban,No,NA,NA,⋯,NA,Unsure,NA,No,NA,No,NA,NA,NA,Complete
aadegbulugbe@totalhealthcare.org,222,Federally Qualified Health Center (FQHC),NA,Maryland,21223,Urban,Yes,1,1,⋯,NA,Yes,increases stress due to more stigma and discrimination,No,NA,No,NA,NA,"Major issue is with insurance coverage for medications which has no rhyme or reason and significantly disrupts patient care and results in major increase in workload, need coverage protections for ART and PrEP medications returned",Complete
aalbior@bach.health,1550,Federally Qualified Health Center (FQHC),NA,California,94538,Suburban,No,NA,NA,⋯,NA,Yes,NA,No,NA,No,NA,NA,NA,Complete
aalcanta@conemaugh.org,1385,AIDS Service Organization (ASO),NA,Pennsylvania,15905,Rural,No,1,1,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,Incomplete
abby.hinz@eauclairecounty.gov,258,Health department clinic/public health clinic,NA,Wisconsin,54703,Urban,Yes,1,NA,⋯,NA,Yes,Feel less support for the general community/society,No,NA,No,NA,NA,NA,Complete


## Number of incomplete and complete responses among LGBTQ providers

In [103]:
table(LGBTQ_providers$Q819)


  Complete Incomplete 
       754        269 

## Descriptive Statistics for LGBTQ providers

### Age counts among LGBTQ providers

In [104]:
## LGBTQ providers - Age
age_LGBTQ <- table(LGBTQ_providers$Q162)
age_LGBTQ


                18-24                 25-34                 35-44 
                   31                   196                   241 
                45-54                 55-64                    65 
                  243                   202                    88 
Prefer not to respond 
                   19 

### Age percentages among LGBTQ providers

In [105]:
age_perc_LGBTQ <- prop.table(age_LGBTQ)*100
age_perc_LGBTQ


                18-24                 25-34                 35-44 
             3.039216             19.215686             23.627451 
                45-54                 55-64                    65 
            23.823529             19.803922              8.627451 
Prefer not to respond 
             1.862745 

### Gender counts among LGBTQ providers

In [106]:
## LGBTQ providers - Gender
gender_LGBTQ <- table(LGBTQ_providers$Q157)
gender_LGBTQ


                                Female/Woman 
                                         625 
Genderqueer/gender non-confirming/non-binary 
                                          65 
                                    Male/Man 
                                         271 
                       Prefer not to respond 
                                          17 
                     Prefer to self-describe 
                                           9 
                             Transgender man 
                                          19 
                           Transgender woman 
                                          14 

### Gender percentages among LGBTQ providers

In [107]:
gender_perc_LGBTQ <- prop.table(gender_LGBTQ)*100
gender_perc_LGBTQ


                                Female/Woman 
                                  61.2745098 
Genderqueer/gender non-confirming/non-binary 
                                   6.3725490 
                                    Male/Man 
                                  26.5686275 
                       Prefer not to respond 
                                   1.6666667 
                     Prefer to self-describe 
                                   0.8823529 
                             Transgender man 
                                   1.8627451 
                           Transgender woman 
                                   1.3725490 

### Race counts among LGBTQ providers

In [108]:
## LGBTQ providers - Race
race_LGBTQ <- table(LGBTQ_providers$Q159)
race_LGBTQ


        Asian and/or Asian American           Black or African American 
                                 25                                 251 
                        Multiracial    Native American or Alaska Native 
                                 73                                  11 
Native Hawaiian or Pacific Islander               Prefer not to respond 
                                  5                                  53 
            Prefer to self-describe                  White or Caucasian 
                                 24                                 577 

### Race percentages among LGBTQ providers

In [109]:
race_perc_LGBTQ <- prop.table(race_LGBTQ)*100
race_perc_LGBTQ


        Asian and/or Asian American           Black or African American 
                          2.4533857                          24.6319921 
                        Multiracial    Native American or Alaska Native 
                          7.1638862                           1.0794897 
Native Hawaiian or Pacific Islander               Prefer not to respond 
                          0.4906771                           5.2011776 
            Prefer to self-describe                  White or Caucasian 
                          2.3552502                          56.6241413 

### Hispanic/Latinx percentages among LGBTQ providers

In [110]:
## LGBTQ providers - Hispanic/Latinx
hisp_LGBTQ <- table(LGBTQ_providers$Q161)
hisp_LGBTQ


                   No Prefer not to respond                   Yes 
                  814                    19                   187 

### Hispanic/Latinx percentages among LGBTQ providers

In [111]:
hisp_perc_LGBTQ <- prop.table(hisp_LGBTQ)*100
hisp_perc_LGBTQ 


                   No Prefer not to respond                   Yes 
            79.803922              1.862745             18.333333 

### Role counts among LGBTQ providers

In [112]:
role_counts_LGBTQ <- table(LGBTQ_providers$Q66)
role_counts_LGBTQ


                             Dentist      Disease Intervention Specialist 
                                   2                                    1 
     Licensed Clinical Social Worker             Licensed Practical Nurse 
                                  39                                    7 
                   Medical Assistant           Mental Health Case Manager 
                                  10                                    3 
             Mental Health Counselor           Mental Health Professional 
                                   1                                   11 
         Mental Health Professional                         Nurse Manager 
                                   1                                   17 
                       Nurse Midwife                   Nurse Practitioner 
                                   1                                   68 
Other Provider/Clinical professional                           Pharmacist 
                        

### Role percentages among LGBTQ providers

In [113]:
role_perc_LGBTQ <- prop.table(role_counts_LGBTQ)*100
role_perc_LGBTQ


                             Dentist      Disease Intervention Specialist 
                           0.5970149                            0.2985075 
     Licensed Clinical Social Worker             Licensed Practical Nurse 
                          11.6417910                            2.0895522 
                   Medical Assistant           Mental Health Case Manager 
                           2.9850746                            0.8955224 
             Mental Health Counselor           Mental Health Professional 
                           0.2985075                            3.2835821 
         Mental Health Professional                         Nurse Manager 
                           0.2985075                            5.0746269 
                       Nurse Midwife                   Nurse Practitioner 
                           0.2985075                           20.2985075 
Other Provider/Clinical professional                           Pharmacist 
                        

### Prescriber counts among LGBTQ providers

In [114]:
prescriber_counts_LGBTQ <- table(LGBTQ_providers$Q70)
prescriber_counts_LGBTQ


 No Yes 
193 146 

### Prescriber percentages among LGBTQ providers

In [115]:
prescriber_perc_LGBTQ <- prop.table(prescriber_counts_LGBTQ)*100
prescriber_perc_LGBTQ


      No      Yes 
56.93215 43.06785 

## HIV training statistcis for LGBTQ providers

In [116]:
# LGBTQ providers - Create a vector of column names representing the HIV topics columns
hiv_topics_LGBTQ <- c("Q198", "Q199", "Q200", "Q201", "Q202","Q203","Q204","Q205","Q206","Q207","Q208","Q209","Q210a","Q210b")

### HIV topic with greatest number of 1st rankings among LGBTQ providers

In [117]:
# LGBTQ providers - Calculate the counts of 1st rankings for each HIV topic
hiv1_count_LGBTQ <- sapply(hiv_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 1, na.rm = TRUE))
hiv1_count_LGBTQ

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
  178    80    72    17    21    37    34    49    48     6    12    12     3 
Q210b 
    1

In [118]:
# LGBTQ providers - Find the HIV topic with the highest count of 1st rankings
hiv1_rank_LGBTQ <- hiv_topics_LGBTQ[which.max(hiv1_count_LGBTQ)]
hiv1_rank_LGBTQ

[1] "Q198"

In [119]:
# LGBTQ providers - Calculate the percentage of 1st rankings
hiv1_perc_LGBTQ <- (hiv1_count_LGBTQ[which.max(hiv1_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, hiv_topics_LGBTQ])) > 0)) * 100
hiv1_perc_LGBTQ

Q198 
30.95652

### HIV topic with greatest number of 2nd rankings among LGBTQ providers

In [120]:
# LGBTQ providers - Calculate the counts of 2nd rankings for each HIV topic
hiv2_count_LGBTQ <- sapply(hiv_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 2, na.rm = TRUE))
hiv2_count_LGBTQ

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
   56    87    76    26    42    51    48    49    56    15    19     2     0 
Q210b 
    1

In [121]:
# LGBTQ providers - Find the HIV topic with the highest count of 2nd rankings
hiv2_rank_LGBTQ <- hiv_topics_LGBTQ[which.max(hiv2_count_LGBTQ)]
hiv2_rank_LGBTQ

[1] "Q199"

In [122]:
# LGBTQ providers - Calculate the percentage of 2nd rankings
hiv2_perc_LGBTQ  <- (hiv2_count_LGBTQ[which.max(hiv2_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, hiv_topics_LGBTQ])) > 0)) * 100
hiv2_perc_LGBTQ 

Q199 
15.13043

### HIV topic with greatest number of 3rd rankings among LGBTQ providers

In [123]:
# LGBTQ providers - Calculate the counts of 3rd rankings for each HIV topic
hiv3_count_LGBTQ  <- sapply(hiv_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 3, na.rm = TRUE))
hiv3_count_LGBTQ

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
   65    47    53    17    56    46    57    58    52    12    36    16     2 
Q210b 
    3

In [124]:
# LGBTQ providers - Find the PrEP topic with the highest count of 3rd rankings
hiv3_rank_LGBTQ <- hiv_topics_LGBTQ[which.max(hiv3_count_LGBTQ)]
hiv3_rank_LGBTQ

[1] "Q198"

In [125]:
# LGBTQ providers - Calculate the percentage of 3rd ranking
hiv3_perc_LGBTQ <- (hiv3_count_LGBTQ[which.max(hiv3_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, hiv_topics_LGBTQ])) > 0)) * 100
hiv3_perc_LGBTQ

Q198 
11.30435

### Top 3 HIV topics among LGBTQ providers

In [126]:
# Calculate the counts for each HIV topic
hiv_topic_counts_LGBTQ <- sapply(hiv_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] %in% c(1, 2, 3), na.rm = TRUE))
hiv_topic_counts_LGBTQ
# Sort the topics based on their counts in descending order
hiv_sorted_topics_LGBTQ <- sort(hiv_topic_counts_LGBTQ, decreasing = TRUE)
hiv_sorted_topics_LGBTQ
# Get the top 3 topics
top3_hiv_topics_LGBTQ <- names(hiv_sorted_topics_LGBTQ)[1:3]

top3_hiv_topics_LGBTQ

Q198  Q199  Q200  Q201  Q202  Q203  Q204  Q205  Q206  Q207  Q208  Q209 Q210a 
  299   214   201    60   119   134   139   156   156    33    67    30     5 
Q210b 
    5

Q198  Q199  Q200  Q205  Q206  Q204  Q203  Q202  Q208  Q201  Q207  Q209 Q210a 
  299   214   201   156   156   139   134   119    67    60    33    30     5 
Q210b 
    5

[1] "Q198" "Q199" "Q200"

### Percentage of top 3 HIV topics among LGBTQ providers

In [127]:
# Calculate the counts for each ranking (1, 2, 3) for the top 3 topics
hiv_ranking_counts_LGBTQ <- apply(LGBTQ_providers[top3_hiv_topics_LGBTQ], 1, function(row) any(row %in% c(1, 2, 3)))

# Calculate the total count of rows where one of the top 3 topics was ranked as 1, 2, or 3
hiv_total_count_LGBTQ <- sum(hiv_ranking_counts_LGBTQ)

# Calculate the total count of respondents who provided a ranking (1, 2, or 3) for any of the HIV topic questions
hiv_total_respondents_LGBTQ <- sum(apply(LGBTQ_providers[hiv_topics_LGBTQ], 1, function(row) any(row %in% c(1, 2, 3))))

# Calculate the percentage of rows where one of the top 3 topics was ranked as 1, 2, or 3 among the respondents who provided rankings
hiv_total_percentage_LGBTQ <- (hiv_total_count_LGBTQ / hiv_total_respondents_LGBTQ) * 100
hiv_total_percentage_LGBTQ

[1] 81.04348

## PrEP training statistcis for LGBTQ providers

In [128]:
# LGBTQ providers - Create a vector of column names representing the PrEP topics columns
prep_topics_LGBTQ <- c("Q211", "Q212", "Q213", "Q214", "Q215","Q216","Q217","Q218","Q219","Q220","Q221","Q222","Q223","Q224","Q225")

### PrEP topic with greatest number of 1st rankings among LGBTQ providers

In [129]:
# LGBTQ providers - Calculate the counts of 1st rankings for each PrEP topic
prep1_count_LGBTQ <- sapply(prep_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 1, na.rm = TRUE))
prep1_count_LGBTQ

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
 124   73   62  108   11   22   20   48   37   31   19   29   36   12    3

In [130]:
# LGBTQ providers - Find the PrEP topic with the highest count of 1st rankings
prep1_rank_LGBTQ <- prep_topics_LGBTQ[which.max(prep1_count_LGBTQ)]
prep1_rank_LGBTQ

[1] "Q211"

In [131]:
# LGBTQ providers - Calculate the percentage of 1st rankings
prep1_perc_LGBTQ <- (prep1_count_LGBTQ[which.max(prep1_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, prep_topics_LGBTQ])) > 0)) * 100
prep1_perc_LGBTQ

Q211 
19.2846

### PrEP topic with greatest number of 2nd rankings among LGBTQ providers

In [132]:
# LGBTQ providers - Calculate the counts of 2nd rankings for each PrEP topic
prep2_count_LGBTQ <- sapply(prep_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 2, na.rm = TRUE))
prep2_count_LGBTQ

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
  29   70   53   80   26   34   37   77   50   39   24   27   41   15    0

In [133]:
# LGBTQ providers - Find the PrEP topic with the highest count of 2nd rankings
prep2_rank_LGBTQ <- prep_topics_LGBTQ[which.max(prep2_count_LGBTQ)]
prep2_rank_LGBTQ

[1] "Q214"

In [134]:
# LGBTQ providers - Calculate the percentage of 2nd rankings
prep2_perc_LGBTQ  <- (prep2_count_LGBTQ[which.max(prep2_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, prep_topics_LGBTQ])) > 0)) * 100
prep2_perc_LGBTQ 

Q214 
12.44168

### PrEP topic with greatest number of 3rd rankings among LGBTQ providers

In [135]:
# LGBTQ providers - Calculate the counts of 3rd rankings for each PrEP topic
prep3_count_LGBTQ  <- sapply(prep_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 3, na.rm = TRUE))
prep3_count_LGBTQ

Q211 Q212 Q213 Q214 Q215 Q216 Q217 Q218 Q219 Q220 Q221 Q222 Q223 Q224 Q225 
  44   21   43   52   18   35   38   69   26   66   42   46   42   41    5

In [136]:
# LGBTQ providers - Find the PrEP topic with the highest count of 3rd rankings
prep3_rank_LGBTQ <- prep_topics_LGBTQ[which.max(prep3_count_LGBTQ )]
prep3_rank_LGBTQ

[1] "Q218"

In [137]:
# LGBTQ providers - Calculate the percentage of 3rd ranking
prep3_perc_LGBTQ <- (prep3_count_LGBTQ[which.max(prep3_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, prep_topics_LGBTQ])) > 0)) * 100
prep3_perc_LGBTQ

Q218 
10.73095

### Top 3 PrEP topics among LGBTQ providers

In [138]:
# Calculate the counts for each PrEP topic
prep_topic_counts_LGBTQ <- sapply(prep_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] %in% c(1, 2, 3), na.rm = TRUE))

# Sort the topics based on their counts in descending order
prep_sorted_topics_LGBTQ <- sort(prep_topic_counts_LGBTQ, decreasing = TRUE)

# Get the top 3 topics
top3_prep_topics_LGBTQ <- names(prep_sorted_topics_LGBTQ)[1:3]
top3_prep_topics_LGBTQ

[1] "Q214" "Q211" "Q218"

### Percentage of top 3 PrEP topics

In [139]:
# Calculate the counts for each ranking (1, 2, 3) for the top 3 topics
prep_ranking_counts_LGBTQ <- apply(LGBTQ_providers[top3_prep_topics_LGBTQ], 1, function(row) any(row %in% c(1, 2, 3)))

# Calculate the total count of rows where one of the top 3 topics was ranked as 1, 2, or 3
prep_total_count_LGBTQ <- sum(prep_ranking_counts_LGBTQ)

# Calculate the total count of respondents who provided a ranking (1, 2, or 3) for any of the HIV topic questions
prep_total_respondents_LGBTQ <- sum(apply(LGBTQ_providers[prep_topics_LGBTQ], 1, function(row) any(row %in% c(1, 2, 3))))

# Calculate the percentage of rows where one of the top 3 topics was ranked as 1, 2, or 3 among the respondents who provided rankings
prep_total_percentage_LGBTQ <- (prep_total_count_LGBTQ / prep_total_respondents_LGBTQ) * 100
prep_total_percentage_LGBTQ 

[1] 73.87247

## LGBTQ training statistcis for LGBTQ providers

In [140]:
# Create a vector of column names representing the LGBTQ topics columns
lgbtq_topics_LGBTQ <- c("Q227", "Q228", "Q229", "Q230", "Q231","Q232","Q233","Q234","Q235")

### LGBTQ topic with greatest number of 1st rankings among LGBTQ providers

In [141]:
# LGBTQ providers - Calculate the counts of 1st rankings for each LGBTQ topic
lgbtq1_count_LGBTQ <- sapply(lgbtq_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 1, na.rm = TRUE))
lgbtq1_count_LGBTQ

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
 132  178  114  111   29   58  110  108   14

In [142]:
# LGBTQ providers - Find the LGBTQ topic with the highest count of 1st rankings
lgbtq1_rank_LGBTQ <- lgbtq_topics_LGBTQ[which.max(lgbtq1_count_LGBTQ)]
lgbtq1_rank_LGBTQ

[1] "Q228"

In [143]:
# LGBTQ providers - Calculate the percentage of 1st rankings
lgbtq1_perc_LGBTQ <- (lgbtq1_count_LGBTQ[which.max(lgbtq1_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, lgbtq_topics_LGBTQ])) > 0)) * 100
lgbtq1_perc_LGBTQ

Q228 
20.57803

### LGBTQ topic with greatest number of 2nd rankings among LGBTQ providers

In [144]:
# LGBTQ providers - Calculate the counts of 2nd rankings for each LGBTQ topic
lgbtq2_count_LGBTQ <- sapply(lgbtq_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 2, na.rm = TRUE))
lgbtq2_count_LGBTQ

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
  62  113  101  137   60   71  128  128    3

In [145]:
# LGBTQ providers - Find the LGBTQ topic with the highest count of 2nd rankings
lgbtq2_rank_LGBTQ <- lgbtq_topics_LGBTQ[which.max(lgbtq2_count_LGBTQ)]
lgbtq2_rank_LGBTQ

[1] "Q230"

In [146]:
# LGBTQ providers - Calculate the percentage of 2nd rankings
lgbtq2_perc_LGBTQ <- (lgbtq2_count_LGBTQ[which.max(lgbtq2_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, lgbtq_topics_LGBTQ])) > 0)) * 100
lgbtq2_perc_LGBTQ

Q230 
15.83815

### LGBTQ topic with greatest number of 3rd rankings among LGBTQ providers

In [147]:
# LGBTQ providers - Calculate the counts of 3rd rankings for each LGBTQ topic
lgbtq3_count_LGBTQ <- sapply(lgbtq_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] == 2, na.rm = TRUE))
lgbtq3_count_LGBTQ

Q227 Q228 Q229 Q230 Q231 Q232 Q233 Q234 Q235 
  62  113  101  137   60   71  128  128    3

In [148]:
# LGBTQ providers - Find the LGBTQ topic with the highest count of 2nd rankings
lgbtq3_rank_LGBTQ <- lgbtq_topics_LGBTQ[which.max(lgbtq3_count_LGBTQ)]
lgbtq3_rank_LGBTQ

[1] "Q230"

In [149]:
# LGBTQ providers - Calculate the percentage of 2nd rankings
lgbtq3_perc_LGBTQ <- (lgbtq3_count_LGBTQ[which.max(lgbtq3_count_LGBTQ)] / sum(rowSums(!is.na(LGBTQ_providers[, lgbtq_topics_LGBTQ])) > 0)) * 100
lgbtq3_perc_LGBTQ

Q230 
15.83815

### Top 3 LGBTQ topics among LGBTQ providers

In [150]:
# Calculate the counts for each LGBTQ topic
lgbtq_topic_counts_LGBTQ <- sapply(lgbtq_topics_LGBTQ, function(topic) sum(LGBTQ_providers[[topic]] %in% c(1, 2, 3), na.rm = TRUE))

# Sort the topics based on their counts in descending order
lgbtq_sorted_topics_LGBTQ <- sort(lgbtq_topic_counts_LGBTQ, decreasing = TRUE)

# Get the top 3 topics
top3_lgbtq_topics_LGBTQ <- names(lgbtq_sorted_topics_LGBTQ)[1:3]
top3_lgbtq_topics_LGBTQ

[1] "Q234" "Q228" "Q230"

### Percentage of top 3 LGBTQ topics among LGBTQ providers

In [151]:
# Calculate the counts for each ranking (1, 2, 3) for the top 3 topics
lgbtq_ranking_counts_LGBTQ <- apply(LGBTQ_providers[top3_lgbtq_topics_LGBTQ], 1, function(row) any(row %in% c(1, 2, 3)))

# Calculate the total count of rows where one of the top 3 topics was ranked as 1, 2, or 3
lgbtq_total_count_LGBTQ <- sum(lgbtq_ranking_counts_LGBTQ)

# Calculate the total count of respondents who provided a ranking (1, 2, or 3) for any of the LGBTQ topic questions
lgbtq_total_respondents_LGBTQ <- sum(apply(LGBTQ_providers[lgbtq_topics_LGBTQ], 1, function(row) any(row %in% c(1, 2, 3))))

# Calculate the percentage of rows where one of the top 3 topics was ranked as 1, 2, or 3 among the respondents who provided rankings
lgbtq_total_percentage_LGBTQ <- (lgbtq_total_count_LGBTQ / lgbtq_total_respondents_LGBTQ) * 100
lgbtq_total_percentage_LGBTQ 

[1] 86.5896